In [7]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

## Jordan Form - Real Eigenvalues

This example shows the Jordan Form for the dynamic matrix of Linear Time Invariant (LTI) systems. 
The Jordan form is a particular matrix form that simplifies the modal analysis of a LTI system by highlighting the eigenvalues of the system and the relations between states especially in the case of repeated eigenvalues. 

The Jordan form can be obtained from the dynamic matrix $A$ of the system $\dot x = A x$ as:

$$
J = V^{-1}AV, 
$$

where $V$ is a matrix composed of the, so called, generalized eigenvectors of the eigenvalues of $A$.

Clearly, since $A$ and $J$ are similar matrices, they share the same eigenvalues and also the same system modes. 

A matrix in Jordan form is a block diagonal matrix like:

$$
J = \begin{bmatrix}
J_1 & 0 & 0 & ... \\
0 & J_2 & 0 & ... \\
0 & 0 & J_3 & ... \\
... & ... & ... & ...\\
\end{bmatrix},
$$

where the elements $J_i$ are the so called Jordan mini-blocks that look like:

$$
J_i = \begin{bmatrix}
\lambda_i & 1 & 0 & ... \\
0 & \lambda_i & 1 & ... \\
0 & 0 & \lambda_i & ... \\
... & ... & ... & ...\\
\end{bmatrix},
$$

where $\lambda_i$ is the $i$-th eigenvalue of $J$ (and of $A$). 
The number and dimensions of mini-blocks depends on the multiplicity of the eigenvalues. 

### How to use this notebook?

- Define a matrix and watch its Jordan form; experiment with both real and complex eigenvalues.
- Note how the Jordan form of a system matrix with distinct eigenvalues is actually a diagonal matrix (all Jordan mini-blocks of dimension 1).
- Note the relation between the number and dimension of the Jordan mini-blocks of an eigenvalue and the modes associated with it.
- Note that in the case of complex eigenvalues the matrix $J$ has complex elements on the diagonal and so does the generalized eigenvectors matrix $V$.
- Explore the effects of changing matrix $A$ values or load example matrices. 

In [8]:
#Preparatory Cell 

import control
import numpy
import sympy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

from sympy import Matrix 



#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#create a NxM matrix widget 
def createMatrixWidget(n,m):
    M = widgets.GridBox(children=[widgets.FloatText(layout=widgets.Layout(width='100px', height='40px'),
    value=0.0, disabled=False, label=i) for i in range(n*m)],
    layout=widgets.Layout(
        #width='50%',
        grid_template_columns= ''.join(['100px ' for i in range(m)]),
        #grid_template_rows='80px 80px 80px',
        grid_row_gap='0px',
        track_size='0px')
    )
    return M


#extract matrix from widgets and convert to numpy matrix
def getNumpyMatFromWidget(M,n,m):
    #get W gridbox dims
    M_ = numpy.matrix(numpy.zeros((n,m)))
    for irow in range(0,n):
        for icol in range(0,m):
            M_[irow,icol] = M.children[irow*3+icol].value

            
#this is a simple derived class from FloatText used to experience with interact             
class floatWidget(widgets.FloatText):
    def __init__(self,**kwargs):
        #self.n = n
        self.value = 30.0
        #self.M = 
        widgets.FloatText.__init__(self, **kwargs)

#    def value(self):
#        return 0 #self.FloatText.value

from traitlets import Unicode
from ipywidgets import register 


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

        

        
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass



In [9]:
#define matrices
A = matrixWidget(4,4)

#this is the main callback and does all the computations and plots 
def main_callback(matA,DW,sel):
    #check if a specific matrix is requested or is manual 
    if sel=='manual definition of the system' :
        pass
    elif sel == 'stable system - no complex congjugate poles':
        matA = numpy.zeros((4,4))
        matA[0,0] = -1
        matA[1,1] = -2
        matA[2,2] = -3
        matA[3,3] = -4
        A.setM(matA)
    elif sel == 'stable system - w/ complex conjugate pair':
        matA = numpy.zeros((4,4))
        matA[0,0] = -1
        matA[0,1] = 3
        matA[1,0] = -3
        matA[1,1] = -1
        matA[2,2] = -3
        matA[3,3] = -4
        A.setM(matA)
    elif sel == 'unstable system - unstable real pole unstable':
        matA = numpy.zeros((4,4))
        matA[0,0] = 1
        matA[1,1] = -2
        matA[2,2] = -3
        matA[3,3] = -4
        A.setM(matA)
    elif sel ==  'unstable system - unstable complex conjugate pair ':
        matA = numpy.zeros((4,4))
        matA[0,0] = 1
        matA[0,1] = 3
        matA[1,0] = -3
        matA[1,1] = 1
        matA[2,2] = -3
        matA[3,3] = -4
        A.setM(matA)
    else : 
        matA = numpy.zeros((4,4))
        A.setM(matA)
    
    # Work with symbolic matrix
    matAs = sympy.Matrix(matA)
    dictEig = matAs.eigenvals()
    eigs = list(dictEig.keys())
    algMult = list(dictEig.values())
    
    # check dimension of jordan blocks
    dimJblock = []
    for i in range(len(eigs)):
        dimJblock.append(algMult[i]-len((matAs-eigs[i]*sympy.eye(4)).nullspace())+1)
        
    # jordan form
    matAs_P, matAs_J = matAs.jordan_form(chop=True)
    
    timeVectors = []
    modeVectors = []
    # compute modes simulations and prepare modestring
    modestring = ''
    for i in range(len(eigs)):
        sim = []
        if sympy.re(eigs[i]) >= 0:
            # instable or integral like
            time = numpy.linspace(0,10,1000)
            for n in range(dimJblock[i]):
                if n==0:
                    if sympy.im(eigs[i]) != 0 and (sympy.conjugate(eigs[i]) not in eigs[0:i]):
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time)*numpy.cos(float(sympy.im(eigs[i]))*time))
                        modestring = modestring + "$e^{%s t} cos(%s t + \phi)$  " % (str(float(sympy.re(eigs[i]))), str(float(sympy.im(eigs[i]))))
                    elif sympy.im(eigs[i]) == 0:
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time))
                        modestring = modestring + "$e^{%s t}$  " % (str(float(sympy.re(eigs[i]))))
                else:
                    if sympy.im(eigs[i]) != 0 and (sympy.conjugate(eigs[i]) not in eigs[0:i]):
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time)*numpy.cos(float(sympy.im(eigs[i]))*time))
                        modestring = modestring + "$t^{%s}e^{%s t} cos(%s t + \phi)$  " % (str(n), str(float(sympy.re(eigs[i]))), str(float(sympy.im(eigs[i]))))
                    elif sympy.im(eigs[i]) == 0:
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time))
                        modestring = modestring + "$t^{%s}e^{%s t}$  " % (str(n), str(float(sympy.re(eigs[i]))))
        else:
            # stable mode
            time = numpy.linspace(0,10*(1/float(sympy.Abs(eigs[i]))),1000)
            for n in range(dimJblock[i]):
                if n==0:
                    if sympy.im(eigs[i]) != 0 and (sympy.conjugate(eigs[i]) not in eigs[0:i]):
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time)*numpy.cos(float(sympy.im(eigs[i]))*time))
                        modestring = modestring + "$e^{%s t} cos(%s t + \phi)$  " % (str(float(sympy.re(eigs[i]))), str(float(sympy.im(eigs[i]))))
                    elif sympy.im(eigs[i]) == 0:
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time))
                        modestring = modestring + "$e^{%s t}$  " % (str(float(sympy.re(eigs[i]))))
                else:
                    if sympy.im(eigs[i]) != 0 and (sympy.conjugate(eigs[i]) not in eigs[0:i]):
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time)*numpy.cos(float(sympy.im(eigs[i]))*time))
                        modestring = modestring + "$t^{%s}e^{%s t} cos(%s t + \phi)$  " % (str(n), str(float(sympy.re(eigs[i]))), str(float(sympy.im(eigs[i]))))
                    elif sympy.im(eigs[i]) == 0:
                        sim.append(time**n*numpy.exp(float(sympy.re(eigs[i]))*time))
                        modestring = modestring + "$t^{%s}e^{%s t}$  " % (str(n), str(float(sympy.re(eigs[i]))))
        if len(sim) != 0:
            timeVectors.append(time)
            modeVectors.append(sim)
    
    #print(dimJblock)
    #print(len(modeVectors))
    
    #create textual output            
    display(Markdown('Matrix: $%s$ has eigenvalues $%s$' % (vmatrix(matA), vmatrix(numpy.array(numpy.linalg.eig(matA)[0])))))
    #for better visualization
    matJlist = []
    for i in range(4):
        temp = []
        for j in range(4):
            if sympy.im(matAs_J[i,j]) != 0:
                temp.append(numpy.complex(matAs_J[i,j]))
            else:
                temp.append(numpy.real(matAs_J[i,j]))
        matJlist.append(temp)
    matJ = numpy.matrix(matJlist)
    display(Markdown('and the Jordan form equal to: $%s$' %str(vmatrix(matJ))))
    #for better visualization
    matPlist = []
    for i in range(4):
        temp = []
        for j in range(4):
            if sympy.im(matAs_P[i,j]) != 0:
                temp.append(numpy.complex(matAs_P[i,j]))
            else:
                temp.append(numpy.real(matAs_P[i,j]))
        matPlist.append(temp)
    matP = numpy.matrix(matPlist)
    display(Markdown('with generalized eigenvectors $%s$.' %str(vmatrix(matP))))
    display(Markdown('The modes are: %s' % modestring))
    
    #compute total number of figures
    totfig=0
    for i in range(len(modeVectors)):
            totfig = totfig + len(modeVectors[i])
            
    #plot each single mode
    fig = plt.figure(figsize=(20, 4))
    idx = 1
    for i in range(len(timeVectors)):
        for j in range(len(modeVectors[i])):
            sf = fig.add_subplot(1,totfig,idx)
            idx = idx + 1
            sf.plot(timeVectors[i],modeVectors[i][j])
            sf.grid(True)
            plt.xlabel(r'$t$ [s]')
            plt.axvline(x=0,color='black',linewidth=0.8)
            plt.axhline(y=0,color='black',linewidth=0.8)

    
    
#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

#define type of ipout 
SELECT = widgets.Dropdown(
    options=['manual definition of the system', 'reset', 'stable system - no complex congjugate poles', 
             'stable system - w/ complex conjugate pair', 
             'unstable system - unstable real pole unstable', 
             'unstable system - unstable complex conjugate pair '],
    value='manual definition of the system',
    description='Examples:',
    disabled=False,
)


#create a graphic structure to hold all widgets 
alltogether =  widgets.VBox([SELECT, widgets.Label(''), widgets.HBox([widgets.Label('$\dot{x}(t) = $',border=3), A,widgets.Label('$x(t)$',border=3), START])] )
    

out = widgets.interactive_output(main_callback,{'matA': A, 'DW': DW, 'sel': SELECT})
out.layout.height = '600px'
display(alltogether,out)

Output(layout=Layout(height='600px'))